In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import collections
import scipy

from scipy.spatial import KDTree
from scipy.spatial import cKDTree
from sklearn.neighbors import BallTree

In this homework your task will be to modify `KNNClassifier` class from your practice in class. The class with empty methods is provided below. Please, modify it to do all tasks.

In [2]:
class KNNClassifier(object):
    
    def __init__(self, k_neighbors=1, max_dist=1.):
        """
        This is a constructor of the class. 
        Here you can define parameters (max_dist) of the class and 
        attributes, that are visible within all methods of the class.
        
        Parameters
        ----------
        k_neighbors : int
            Number of neighbors used for classification.
        max_dist : float
            Maximum distance between an object and its neighbors.
        """
        
        # Make this parameter visible in all methods of the class
        self.k_neighbors = k_neighbors
        self.max_dist = max_dist
        
        self.X_fit = None
        self.y_fit = None
                
    
    def fit(self, X, y):
        """
        This method trains the KNN classifier. 
        Actualy, the KNN classifier has no training procedure.
        It just remembers data (X, y) that will be used for predictions.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
        y : numpy.array, shape = (n_objects)
            1D array with the object labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        self.X_fit = X
        self.y_fit = y
        
        pass
        
    def predict(self, X, method='standard', count_dist = False):
        """
        This methods performs labels prediction for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted : numpy.array, shape = (n_objects)
            1D array with predicted labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        # I saw the predict from seminar but I have my own idea of how it should look. Hope it's not a problem :)
        if method == 'standard':
            # Create an empty list of probabilities of each class
            y_predicted_proba = np.zeros(shape=(X.shape[0], len(np.unique(self.y_fit))))
            for i in range(X.shape[0]):
                for j in range(len(self.X_fit)):
                    dist = np.sqrt(np.sum((self.X_fit[j] - X[i])**2))
                    if dist <= self.max_dist:
                        if count_dist:
                            y_predicted_proba[i][self.y_fit[j]] += 1./dist
                        else:
                            y_predicted_proba[i][self.y_fit[j]] += 1
                    
                if y_predicted_proba[i].sum() > 0:
                    y_predicted_proba[i] /= y_predicted_proba[i].sum()        
                else:
                    for j in range(len(y_predicted_proba[i])):
                        y_predicted_proba[i][j] = 1./len(y_predicted_proba[i])
        
            y_predicted = np.ndarray(len(X))
            for i in range(len(X)):
                y_predicted[i] = np.argmax(y_predicted_proba[i])
            
            return y_predicted # return numpy.array
        
        if method == 'kd-tree':
            
            kd_test = KDTree(X)
            
            kd_fit = KDTree(self.X_fit)
            y_res = kd_test.query_ball_tree(kd_fit, self.max_dist)
            
            for i in range(len(y_res)):
                for j in range(len(y_res[i])):
                    y_res[i][j] = self.y_fit[y_res[i][j]]
                y_res[i] = collections.Counter(y_res[i]).most_common(1)[0][0]
                
            return np.array(y_res)  
        
        
        if method == 'ckd-tree':
            
            ckd_test = cKDTree(X)
            
            ckd_fit = cKDTree(self.X_fit)
            y_res = ckd_test.query_ball_tree(ckd_fit, self.max_dist)
        
            for i in range(len(y_res)):
                for j in range(len(y_res[i])):
                    y_res[i][j] = self.y_fit[y_res[i][j]]
                y_res[i] = collections.Counter(y_res[i]).most_common(1)[0][0]
                
            return np.array(y_res)    
        
        if method == 'ball-tree':
            
            tree = BallTree(self.X_fit)
            
            y_res = tree.query_radius(X, self.max_dist)
            
            for i in range(len(y_res)):
                for j in range(len(y_res[i])):
                    y_res[i][j] = self.y_fit[y_res[i][j]]
                y_res[i] = np.int64(collections.Counter(y_res[i]).most_common(1)[0][0])
                
            # return np.array(y_res) leads to unknown type and error so i need this :/    
            y_pred = np.ndarray(len(y_res))
            for i in range(y_pred.shape[0]):
                y_pred[i] = y_res[i]
            return y_pred
            
    
    def predict_proba(self, X, count_dist=False):
        """
        This methods performs prediction of probabilities of each class for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted_proba : numpy.array, shape = (n_objects, n_classes)
            2D array with predicted probabilities of each class. 
            Example:
                y_predicted_proba = [[0.1, 0.9],
                                     [0.8, 0.2], 
                                     [0.0, 1.0], 
                                     ...]
        """
        y_predicted_proba = np.zeros(shape=(X.shape[0], len(np.unique(self.y_fit))))
        for i in range(X.shape[0]):
            for j in range(len(self.X_fit)):
                dist = np.sqrt(np.sum((self.X_fit[j] - X[i])**2))
                if dist <= self.max_dist:
                    if count_dist:
                        y_predicted_proba[i][self.y_fit[j]] += 1./dist
                    else:
                        y_predicted_proba[i][self.y_fit[j]] += 1
                    
            if y_predicted_proba[i].sum() > 0:
                y_predicted_proba[i] /= y_predicted_proba[i].sum()        
            else:
                for j in range(len(y_predicted_proba[i])):
                    y_predicted_proba[i][j] = 1./len(y_predicted_proba[i])
                    
            
        return np.array(y_predicted_proba) # return numpy.array

### Task 1 (1 point) <br/>
Create a sample with N=1000 points using `sklearn.datasets.make_moons()` function from scikit-learn library. Also, set up random state in the function `random_state=42`. To open the function description use `Shift` + `Tab` .

In [3]:
from sklearn.datasets import make_moons

sample = make_moons(n_samples=1000, random_state=42, noise=0.3)

### Task 2 (1 point) <br/>

Split the sample into train and test samples using `sklearn.model_selection.train_test_split()` function from scikit-learn library.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sample[0], sample[1], random_state=42, test_size=0.2)

assert y_test.shape[0] == 200

### Task 3 (2 points) <br/>

Modify class `KNNClassifier` above and implement `predict()` method that uses **max_dist** parameter to select neighbors like it's shown in the second figure (radius search). What accuracy do you have for your test sample?

<img src="img/knn2.png" width="600">

In [5]:
# Create a class object
knn = KNNClassifier(max_dist=0.5)

# Train the classifier
knn.fit(X_train, y_train)

In [6]:
%time y_test_predict = knn.predict(X_test)

CPU times: user 7.77 s, sys: 10.9 ms, total: 7.78 s
Wall time: 7.82 s


In [7]:
# Import accuracy_score function
from sklearn.metrics import accuracy_score

# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)

print("Test accuracy of KNN classifier: ", accuracy_test)

Test accuracy of KNN classifier:  0.915


### Task 4 (2 points) <br/>

There are two algorithms [kd-tree](https://en.wikipedia.org/wiki/K-d_tree) and [ball-tree](https://en.wikipedia.org/wiki/Ball_tree) that help to find neighbors very fast. Using [scipy.spatial.KDTree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html) function modify you classifier to speed up **predict** method. Please, create a new hyperparameter of the classifier (similar to k_neighbors) to be able to select algorithm of finding neighbors. Compare speed of predictions with differnet options (kd-tree, ball-tree, just numpy selection).

In [8]:
# Create a class object
knn = KNNClassifier(max_dist=0.5)

# Train the classifier
knn.fit(X_train, y_train)

In [9]:
%time y_test_predict = knn.predict(X_test, method='kd-tree')

CPU times: user 1.26 s, sys: 7.99 ms, total: 1.27 s
Wall time: 1.26 s


In [10]:
accuracy_test = accuracy_score(y_test, y_test_predict)

print("Test accuracy of KNN classifier with KDTree: ", accuracy_test)

Test accuracy of KNN classifier with KDTree:  0.915


In [11]:
# Create a class object
knn = KNNClassifier(max_dist=0.5)

# Train the classifier
knn.fit(X_train, y_train)

In [12]:
%time y_test_predict = knn.predict(X_test, method='ckd-tree')

CPU times: user 118 ms, sys: 62 µs, total: 118 ms
Wall time: 121 ms


In [13]:
accuracy_test = accuracy_score(y_test, y_test_predict)

print("Test accuracy of KNN classifier with cKDTree: ", accuracy_test)

Test accuracy of KNN classifier with cKDTree:  0.915


In [14]:
# Create a class object
knn = KNNClassifier(max_dist=0.5)

# Train the classifier
knn.fit(X_train, y_train)

In [15]:
%time y_test_predict = knn.predict(X_test, method='ball-tree')

CPU times: user 179 ms, sys: 3.4 ms, total: 183 ms
Wall time: 199 ms


In [16]:
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)

print("Test accuracy of KNN classifier with BallTree: ", accuracy_test)

Test accuracy of KNN classifier with BallTree:  0.915


### Task 5 (3 points) <br/>

Now modify the **predict** method to provide prediction with neighbors weights.

<img src="img/wv1.png">

<img src="img/wv2.png">

We propose you to use the following weights:

$$
w_{i} = \frac{1}{\rho(x, x_{i})}
$$

In [17]:
# Create a class object
knn = KNNClassifier(max_dist=0.5)

# Train the classifier
knn.fit(X_train, y_train)

In [18]:
%time y_test_predict = knn.predict(X_test, count_dist=True)

CPU times: user 7.14 s, sys: 3 ms, total: 7.14 s
Wall time: 7.18 s


In [19]:
accuracy_test = accuracy_score(y_test, y_test_predict)

print("Test accuracy of KNN classifier: ", accuracy_test)

Test accuracy of KNN classifier:  0.925


### Task 6 (3 points) <br/>

Develop **predict_proba** method of the classifier. For each object this method returns probability that the object belong to each of the classes. 

For each object $x$ probability for each class is defined as:

$$
p_{c}(x) = \frac{g_{c}(x)}{\sum_{i=1}^{C} g_{i}(x)}
$$

where $C$ is number of classes.

Then, the object has a vector of probabilities:

$$
p(x) = (p_{1}(x), p_{2}(x), ..., p_{C}(x))
$$

In [20]:
# Create a class object
knn = KNNClassifier(max_dist=1)

# Train the classifier
knn.fit(X_train, y_train)

In [21]:
%time y_test_predict_proba = knn.predict_proba(X_test)

# Example of the output
y_test_predict_proba[:10, :] # the first 10 rows

CPU times: user 11.8 s, sys: 50.2 ms, total: 11.9 s
Wall time: 12.1 s


array([[0.19736842, 0.80263158],
       [0.35849057, 0.64150943],
       [0.17973856, 0.82026144],
       [0.86187845, 0.13812155],
       [0.10176991, 0.89823009],
       [0.04268293, 0.95731707],
       [0.30110497, 0.69889503],
       [0.30790191, 0.69209809],
       [0.01980198, 0.98019802],
       [0.53865337, 0.46134663]])

In [22]:
%time y_test_predict_proba = knn.predict_proba(X_test, count_dist=True)

# Example of the output
y_test_predict_proba[:10, :] # the first 10 rows

CPU times: user 13.9 s, sys: 74.1 ms, total: 14 s
Wall time: 14.5 s


array([[0.17016128, 0.82983872],
       [0.45726214, 0.54273786],
       [0.12691127, 0.87308873],
       [0.90435303, 0.09564697],
       [0.06629286, 0.93370714],
       [0.02549551, 0.97450449],
       [0.36387809, 0.63612191],
       [0.3968332 , 0.6031668 ],
       [0.01660806, 0.98339194],
       [0.64129356, 0.35870644]])

I could have done something wrong (and as far as I wrote different from example 'predict' I expect this), but I felt no significant difference in time between all methods. However, they all show similar results, and counting distance in weight even improves prediction a bit. Also I suspect that it depends on sample size - it works just too fast to feel the difference..

P.S. idk what timer says but as for my observation numpy works slower than other methods...
P.P.S. i got the idea that to count time we need to put method on the same line - and it works fine! So, ckdtree and ball tree work great, kd-tree slower and pure numpy (even python) significantly slower. I think that improving my realisation could speed it up, but it will be copypaste on the one side and such experiment seems even more interesting for me..
